In [2]:
# Import statements
import pandas as pd, requests, json
import numpy as np
import math

In [4]:
# Load in the data
loans = pd.read_csv('../All_PPP_Data_150k_plus_080820/All_PPP_Data_150k_plus_080820_0.csv')
loans.head()

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender,CD,ObjectId,longitude,latitude
0,b $2-5 million,NEW HORIZONS TELECOM INC.,901 Cope Industrial Way,PALMER,AK,99645.0,237130.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,64.0,4/10/20 0:00,KeyBank National Association,AK-00,1,-149.093842,61.587723
1,b $2-5 million,"ORTHOALASKA, LLC",3801 Lake Otis Parkway,ANCHORAGE,AK,99508.0,621111.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,240.0,4/4/20 0:00,Northrim Bank,AK-00,2,-149.837771,61.185731
2,b $2-5 million,"PDC, INC",1028 AURORA DR.,FAIRBANKS,AK,99709.0,541330.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,97.0,4/6/20 0:00,Mt. McKinley Bank,AK-00,3,-147.762408,64.855636
3,b $2-5 million,PND ENGINEERS INC,1506 W 36th Ave,ANCHORAGE,AK,99503.0,541330.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,112.0,4/5/20 0:00,Northrim Bank,AK-00,4,-149.911645,61.187860
4,b $2-5 million,PREMIER ALASKA TOURS INC,1900 PREMIER CT,ANCHORAGE,AK,99502.0,561520.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,NaN,5/3/20 0:00,"Wells Fargo Bank, National Association",AK-00,5,-149.920158,61.170583


In [3]:
# Transform panda database into python dictionary
def csvToJson(data, properties, lat="latitude", long="longitude"):
    geojson = {"type":"FeatureCollection", "features":[]}

    for row in data.itertuples():
        feature = {"type":"Feature",
                       "properties":{},
                       "geometry":{"type":"Point",
                                   "coordinates":[]}}

        feature["geometry"]["coordinates"] = [getattr(row, lat),getattr(row, long)]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            element = getattr(row, prop)
            if (isinstance(element, float) and math.isnan(element)):
                feature["properties"][prop] = 'Unanswered'
            else:
                feature["properties"][prop] = element

        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson["features"].append(feature)
        
    return geojson

# Split datatable into array of smaller dataframes by column value
def split_data(data, column_to_split, categories):
    databases = []
    for category in categories:
        file = data.loc[data[column_to_split] == category]
        databases.append(file)
    return databases

In [5]:
columns = ["LoanRange", "BusinessName", "BusinessType", "RaceEthnicity", "Gender", "Lender"]
amounts = ['a $5-10 million', 'b $2-5 million', 'c $1-2 million', 'd $350,000-1 million', 'e $150,000-350,000']

# Split data by loan type
datasets = split_data(loans, 'LoanRange', amounts)

# Output datasets into multiple files
for i, file in enumerate(datasets):
    file_string = 'loans_' + amounts[i][0] + '.json'
    with open(file_string, 'w') as fileout:
        json.dump(csvToJson(file, columns), fileout)

In [4]:
# Combine split datasets into single dictionary with different keys and output a single file
final_dataset = {'a': csvToJson(datasets[0], columns), 'b': csvToJson(datasets[1], columns), 'c': csvToJson(datasets[2], columns), 'd': csvToJson(datasets[3], columns), 'e': csvToJson(datasets[4], columns)}
with open('complete_loan_data.json', 'w') as fileout:
    json.dump(final_dataset, fileout)

In [ ]:
# Output datasets into multiple files
for i, file in enumerate(datasets):
    file_string = 'loans_' + amounts[i][0] + '.json'
    with open(file_string, 'w') as fileout:
        json.dump(csvToJson(file, columns), fileout)

In [11]:
loans.loc[loans['LoanRange'] == "d $350,000-1 million"]


,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender,CD,ObjectId,longitude,latitude
56,"d $350,000-1 million",BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112.0,NaN,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,75.0,5/3/20 0:00,"Bank of America, National Association",WA-07,57,-122.302615,47.636056
57,"d $350,000-1 million",KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407.0,236115.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,21.0,5/3/20 0:00,"Bank of America, National Association",CA-31,58,-117.372547,34.224720
58,"d $350,000-1 million","PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420.0,561730.0,Sole Proprietorship,Unanswered,Male Owned,Non-Veteran,NaN,73.0,5/3/20 0:00,Synovus Bank,SC-01,59,-80.066172,32.942348
59,"d $350,000-1 million","SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150.0,325510.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,62.0,5/3/20 0:00,Synovus Bank,SC-05,60,-80.369893,33.853531
142,"d $350,000-1 million","5150 GLOBAL SOLUTIONS, LLC",271 Park Ave,Wasilla,AK,99654.0,812990.0,Limited Liability Company(LLC),Unanswered,Male Owned,Non-Veteran,NaN,18.0,4/16/20 0:00,"Readycap Lending, LLC",AK-00,143,-149.444640,61.578015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661883,"d $350,000-1 million",WESTERN WYOMING BEVERAGES INC,100 RELIANCE ROAD,ROCK SPRINGS,WY,82901.0,424810.0,Corporation,White,Male Owned,Non-Veteran,NaN,72.0,4/7/20 0:00,RSNB Bank,WY-00,661884,-109.215622,41.661937
661884,"d $350,000-1 million",WESTWARD HEIGHTS CARE CENTER,150 Caring Way,LANDER,WY,82520.0,623110.0,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,103.0,4/27/20 0:00,Central Bank & Trust,WY-00,661885,-108.723050,42.827870
661885,"d $350,000-1 million",WHISLER CHEVROLET CO.,2200 FOOTHILL BLVD,ROCK SPRINGS,WY,82901.0,441110.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,38.0,4/4/20 0:00,NebraskaLand National Bank,WY-00,661886,-109.264434,41.577623
661886,"d $350,000-1 million",WHITE FAMILY ACQUISITION LLC,444 Skyline Drive,GILLETTE,WY,82718.0,441110.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,0.0,4/7/20 0:00,The Huntington National Bank,WY-00,661887,-105.533660,44.290190
